In [1]:
import pandas as pd
import numpy as np
# import scikitplot
import random
import seaborn as sns
import keras
import os
import cv2
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import warnings
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
warnings.simplefilter("ignore")
from keras.models import Model
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
from keras.regularizers import l1, l2
import plotly.express as px
from matplotlib import pyplot as plt


In [2]:
# Define the path to the root folder
train_path = 'images/emotions_dataset/train/'
test_path = 'images/emotions_dataset/test/'
val_path = 'images/emotions_dataset/validation/'


def read_images(path):

    # Initialize a list to store image data as tuples
    image_data = []

    # Create a mapping from string labels (emotions) to class numbers
    emotion_to_class = {'anger': 1, 'fear': 2, 'happiness': 3, 'sadness': 4, 'surprise': 5, 'neutral': 6, 'disgust': 7}

    # Loop through the subfolders
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                if os.path.isfile(image_path) and not image_path.lower().endswith(('.heic', '.HEIC')):
                    # Read the image
                    image = cv2.imread(image_path)                
                    # Extract emotion label from the image name
                    emotion_label = os.path.splitext(image_name)[0]
                    if emotion_label== "surprised":
                        emotion_label= 'surprise'
                    if emotion_label== "sad":
                        emotion_label= 'sadness'
                    # Convert the emotion label to class number using the mapping
                    class_number = emotion_to_class.get(emotion_label.lower(), -1)  # -1 as a default value if the label is not in the mapping
                    
                    # Store image and label as a tuple in the list
                    image_data.append((image, class_number))
    
    return image_data


train_data = read_images(train_path)
test_data = read_images(test_path)
val_data = read_images(val_path)

# Display the length of the collected image data
print("Total train images collected:", len(train_data))
print("Total test images collected:", len(test_data))
print("Total validation images collected:", len(val_data))


Total train images collected: 182
Total test images collected: 70
Total validation images collected: 14


In [3]:
# Define the output directory
train_output_dir = 'images/shuffled/shuffled train data with labels'

test_output_dir = 'images/shuffled/shuffled test data with labels'

val_output_dir = 'images/shuffled/shuffled val data with labels'

# Create a mapping from class numbers to string labels (emotions)
class_to_emotion = {1:'anger', 2:'fear', 3:'happiness',  4:'sadness',  5:'surprise',  6:'neutral', 7:'disgust'}

# Function to save images to directories
def save_images(data, directory):
    for image, emotion in data:
        # Convert class number to emotion label
        emotion_label = class_to_emotion.get(emotion, 'Unknown')
        
        # Create directories based on emotion labels in the output directory
        emotion_dir = os.path.join(directory, emotion_label)
        os.makedirs(emotion_dir, exist_ok=True)

        # Save images to respective emotion directories
        filename = f"{random.randint(0, 1000000)}.jpg"  # Random filename to prevent conflicts
        image_path = os.path.join(emotion_dir, filename)
        cv2.imwrite(image_path, image)

# Save images to the main directory "shuffled data with labels"
save_images(train_data, train_output_dir)
save_images(test_data, test_output_dir)
save_images(val_data, val_output_dir)

KeyboardInterrupt: 

In [ ]:
import dlib
import cv2
import os

def detect_and_save_faces(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize the face detector and landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    # Loop over the images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.JPG')):
            image_path = os.path.join(input_folder, filename)

            # Read the image
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            faces = detector(gray)

            # Loop over the faces and save the extracted faces
            for face in faces:
                landmarks = predictor(gray, face)

                # Draw a rectangle around the face
                cv2.rectangle(img, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

                # Save the extracted face without landmarks
                extracted_face = img[face.top():face.bottom(), face.left():face.right()]
                cv2.imwrite(os.path.join(output_folder, f"{filename}_face.jpg"), extracted_face)

    # Count the number of extracted faces
    num_faces = len(os.listdir(output_folder))
    print(f"Total number of extracted faces: {num_faces}")

In [ ]:
# Define the path to the root folder containing emotion folders
root_folder_path = 'images/shuffled/shuffled train data with labels'

# Loop through each folder of emotions
for folder_name in os.listdir(root_folder_path):
    folder_path = os.path.join(root_folder_path, folder_name)
    if os.path.isdir(folder_path):
        # Define input and output folders for each emotion
        input_folder = folder_path
        output_folder = os.path.join('images/detected faces/train detected faces for each emotion', folder_name)

        # Detect and save faces for the current emotion folder
        detect_and_save_faces(input_folder, output_folder)

Total number of extracted faces: 25
Total number of extracted faces: 26
Total number of extracted faces: 26
Total number of extracted faces: 26
Total number of extracted faces: 26
Total number of extracted faces: 26
Total number of extracted faces: 26


In [ ]:
# Define the path to the root folder containing emotion folders
root_folder_path = 'images/shuffled/shuffled test data with labels'

# Loop through each folder of emotions
for folder_name in os.listdir(root_folder_path):
    folder_path = os.path.join(root_folder_path, folder_name)
    if os.path.isdir(folder_path):
        # Define input and output folders for each emotion
        input_folder = folder_path
        output_folder = os.path.join('images/detected faces/test data detected faces for each emotion', folder_name)

        # Detect and save faces for the current emotion folder
        detect_and_save_faces(input_folder, output_folder)

Total number of extracted faces: 10
Total number of extracted faces: 10
Total number of extracted faces: 10
Total number of extracted faces: 10
Total number of extracted faces: 10
Total number of extracted faces: 10
Total number of extracted faces: 10


In [ ]:
# Define the path to the root folder containing emotion folders
root_folder_path = 'images/shuffled/shuffled val data with labels'

# Loop through each folder of emotions
for folder_name in os.listdir(root_folder_path):
    folder_path = os.path.join(root_folder_path, folder_name)
    if os.path.isdir(folder_path):
        # Define input and output folders for each emotion
        input_folder = folder_path
        output_folder = os.path.join('images/detected faces/validation detected faces for each emotion', folder_name)

        # Detect and save faces for the current emotion folder
        detect_and_save_faces(input_folder, output_folder)

Total number of extracted faces: 2
Total number of extracted faces: 2
Total number of extracted faces: 2
Total number of extracted faces: 2
Total number of extracted faces: 2
Total number of extracted faces: 2
Total number of extracted faces: 2


In [ ]:
def preprocess_images(root_folder, target_size):
    # Get the list of emotions (subfolder names) within the root folder
    emotions = [emotion for emotion in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, emotion))]

    # Create the 'preprocessed' directory to save processed images
    preprocessed_root = os.path.join(root_folder, 'preprocessed')
    os.makedirs(preprocessed_root, exist_ok=True)

    # Loop through each emotion folder
    for emotion in emotions:
        emotion_folder = os.path.join(root_folder, emotion)
        preprocessed_emotion_folder = os.path.join(preprocessed_root, emotion)
        os.makedirs(preprocessed_emotion_folder, exist_ok=True)

        # Get the list of image files in the emotion folder
        image_files = [file for file in os.listdir(emotion_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Preprocess each image in the emotion folder
        for image_file in image_files:
            img_path = os.path.join(emotion_folder, image_file)
            img = cv2.imread(img_path)

            if img is not None:
                # Resize the image
                img_resized = cv2.resize(img, target_size)

                # Convert the image to grayscale
                img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

                # Save the preprocessed image in the preprocessed emotion folder
                new_img_path = os.path.join(preprocessed_emotion_folder, image_file)
                cv2.imwrite(new_img_path, img_gray)
            else:
                print(f"Unable to read image: {image_file}")


In [ ]:
train_root_directory = 'images/detected faces/train detected faces for each emotion'
test_root_directory = 'images/detected faces/test detected faces for each emotion'
val_root_directory = 'images/detected faces/validation detected faces for each emotion'
target_size = (128, 128)
preprocess_images(train_root_directory, target_size)
preprocess_images(test_root_directory, target_size)
preprocess_images(val_root_directory, target_size)

In [ ]:
def augment_images(input_folder, output_folder, target_num_images):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]

    # Check if there are enough images for augmentation
    if len(image_files) == 0:
        print("No images found in the input folder.")
        return

    # Calculate the number of augmentations needed per image
    augmentations_per_image = max(target_num_images // len(image_files), 1)

    # Keep augmenting until the desired number is reached
    augmented_images = []
    for i, image_file in enumerate(image_files):
        # Read the original image
        img = cv2.imread(os.path.join(input_folder, image_file))

        for j in range(augmentations_per_image):
            # Perform augmentations
            flipped_horizontally = cv2.flip(img, 1)
            flipped_hf = cv2.flip(flipped_horizontally, 1)
            # Rotate 30 degrees to the left
            rotation_matrix_left = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), 30, 1)
            rotated_left = cv2.warpAffine(img, rotation_matrix_left, (img.shape[1], img.shape[0]))

            # Rotate 30 degrees to the right
            rotation_matrix_right = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), -30, 1)
            rotated_right = cv2.warpAffine(img, rotation_matrix_right, (img.shape[1], img.shape[0]))

            # Rotate 15 degrees to the left
            rotation_matrix_left_15 = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), 15, 1)
            rotated_left_15 = cv2.warpAffine(img, rotation_matrix_left_15, (img.shape[1], img.shape[0]))

            # Rotate 15 degrees to the right
            rotation_matrix_right_15 = cv2.getRotationMatrix2D((img.shape[1] / 2, img.shape[0] / 2), -15, 1)
            rotated_right_15 = cv2.warpAffine(img, rotation_matrix_right_15, (img.shape[1], img.shape[0]))


            # Add augmented images to the list
            augmented_images.extend([
                flipped_horizontally, flipped_hf,rotated_left,rotated_right,rotated_left_15,rotated_right_15
            ])

    # Randomly shuffle the augmented images
    #random.shuffle(augmented_images)

    # Save only the required number of images
    for i, augmented_image in enumerate(augmented_images[:target_num_images]):
        cv2.imwrite(os.path.join(output_folder, f"{i}_augmented.jpg"), augmented_image)

    print(f"{target_num_images} images requested, {len(augmented_images)} images augmented and saved to {output_folder}")


In [ ]:
train_input_folder = "images/detected faces/train detected faces for each emotion/preprocessed/surprise"
train_output_folder = "images/Augmentation/train/surprise"
test_input_folder = "images/detected faces/test detected faces for each emotion/preprocessed/surprise"
test_output_folder = "images/Augmentation/test/surprise"
train_target_num_images = 416
test_target_num_images = 160

augment_images(train_input_folder, train_output_folder, train_target_num_images)
augment_images(test_input_folder, test_output_folder, test_target_num_images)


160 images requested, 960 images augmented and saved to images/Augmentation/test/sadness
